### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
total_budget = school_data["budget"].sum()
total_budget = f"${total_budget:,.2f}"
total_budget

'$24,649,428.00'

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
school_num = school_data_complete["school_name"].nunique()
stu_num = school_data_complete["Student ID"].nunique()

total_budget = school_data["budget"].sum()


avg_math_score = school_data_complete["math_score"].mean()

avg_read_score = school_data_complete["reading_score"].mean()

avg_pass_score = (avg_math_score+avg_read_score)/2

math_pass_num = school_data_complete.loc[school_data_complete["math_score"] >= 70].count()["math_score"]
math_pass_rate = math_pass_num/stu_num *100

read_pass_num = school_data_complete.loc[school_data_complete["reading_score"] >= 70].count()["reading_score"]
read_pass_rate = read_pass_num/stu_num*100

overall_avg_score = (avg_math_score+avg_read_score)/2

overall_pass_rate = (math_pass_rate + read_pass_rate)/2

stu_num = f"{stu_num:,}"
total_budget = f"${total_budget:,.2f}"
avg_math_score = f"{avg_math_score:.2f}"
avg_read_score = f"{avg_read_score:.2f}"
overall_avg_score = f"{overall_avg_score:.2f}"
math_pass_rate = f"{math_pass_rate:.2f}%"
read_pass_rate = f"{read_pass_rate:.2f}%"
overall_pass_rate = f"{overall_pass_rate:.2f}%"


district_summary = pd.DataFrame({"Total Schools":[school_num],"Total Student":[stu_num],"Total Budget":[total_budget],
                                 "Average Math Schore":[avg_math_score],"Average Reading Score":[avg_read_score],
                                 "Overall Average Score":[overall_avg_score],"% Passing Math":[math_pass_rate],
                                 "% Passing Read":[read_pass_rate],"% Overall Passing":overall_pass_rate})
district_summary

,Total Schools,Total Student,Total Budget,Average Math Schore,Average Reading Score,Overall Average Score,% Passing Math,% Passing Read,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,80.43,74.98%,85.81%,80.39%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
school_data["budget per stu"] = school_data["budget"]/school_data["size"]

stu_groupby_school = student_data.groupby(["school_name"])
avg_math_score_list = stu_groupby_school["math_score"].mean()
avg_reading_score_list = stu_groupby_school["reading_score"].mean()

#Calculate math and reading passing rates per school, and overall pass rate per school
math_pass_count = stu_groupby_school["math_score"].apply(lambda x:(x>=70).sum())
stu_size = stu_groupby_school["Student ID"].count()
school_math_pass = math_pass_count/stu_size*100

read_pass_count = stu_groupby_school["reading_score"].apply(lambda y:(y>=70).sum())
school_read_pass = read_pass_count/stu_size*100

school_overall_pass = (school_math_pass + school_read_pass)/2

#combine data to one dataframe
school_data2 = pd.DataFrame({"Average Math Score":avg_math_score_list,"Average Reading Score":avg_reading_score_list,
                            "% Passing Math":school_math_pass,"% Passing Reading":school_read_pass,
                            "Overall Passing Rate":school_overall_pass})

school_summary = pd.merge(school_data,school_data2, on = "school_name")

school_summary = school_summary.rename(columns = {"school_name":"School Name","type":"School Type","size":"Total Students",
                                                  "budget":"Total School Budget","budget per stu":"Per Student Budget"})

school_summary1 = school_summary.copy()

#format data
school_summary1["Total Students"] = school_summary1["Total Students"].map("{:,}".format)
school_summary1["Total School Budget"] = school_summary1["Total School Budget"].map("${:,}".format)
school_summary1["Per Student Budget"] = school_summary1["Per Student Budget"].map("${:,.2f}".format)
school_summary1["Average Math Score"] = school_summary1["Average Math Score"].map("{:.2f}".format)
school_summary1["Average Reading Score"] = school_summary1["Average Reading Score"].map("{:.2f}".format)
school_summary1["% Passing Math"] = school_summary1["% Passing Math"].map("{:.2f}%".format)
school_summary1["% Passing Reading"] = school_summary1["% Passing Reading"].map("{:.2f}%".format)
school_summary1["Overall Passing Rate"] = school_summary1["Overall Passing Rate"].map("{:.4f}%".format)

school_summary1 = school_summary1.drop("School ID",axis = 1)
school_summary1 = school_summary1.set_index("School Name")
school_summary1.head()


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.5002%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.3639%
Shelton High School,Charter,"1,761","$1,056,600",$600.00,83.36,83.73,93.87%,95.85%,94.8609%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.75%,80.86%,73.8080%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.2657%


# Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
sorted_school_summary = school_summary1.sort_values(["Overall Passing Rate"],ascending = False)
sorted_school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.5867%
Thomas High School,Charter,"1,635","$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.2905%
Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.2703%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.2657%
Wilson High School,Charter,"2,283","$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.2037%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
sorted_school_summary2 = school_summary1.sort_values(["Overall Passing Rate"])
sorted_school_summary2.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,73.2933%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.3639%
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.5002%
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.6400%
Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.8043%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
grade_9th = student_data.loc[student_data["grade"] == "9th"]
grade9_group = grade_9th.groupby(["school_name"])
grade9_math = grade9_group["math_score"].mean()

grade_10th = student_data.loc[student_data["grade"] == "10th"]
grade10_group = grade_10th.groupby(["school_name"])
grade10_math = grade10_group["math_score"].mean()

grade_11th = student_data.loc[student_data["grade"] == "11th"]
grade11_group = grade_11th.groupby(["school_name"])
grade11_math = grade11_group["math_score"].mean()

grade_12th = student_data.loc[student_data["grade"] == "12th"]
grade12_group = grade_12th.groupby(["school_name"])
grade12_math = grade12_group["math_score"].mean()

math_score_grade_sum = pd.DataFrame({"9th Grade":grade9_math,"10th Grade":grade10_math,"11th Grade":grade11_math,
                                    "12th Grade":grade12_math})

math_score_grade_sum.style.format("{:.2f}")

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:

grade9_read = grade9_group["reading_score"].mean()

grade10_read = grade10_group["reading_score"].mean()

grade11_read = grade11_group["reading_score"].mean()

grade12_read = grade12_group["reading_score"].mean()

read_score_grade_sum = pd.DataFrame({"9th Grade":grade9_read,"10th Grade":grade10_read,"11th Grade":grade11_read,
                                    "12th Grade":grade12_read})

read_score_grade_sum.style.format("{:.2f}")

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [10]:
school_summary["budget_group"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels = group_names)

school_spending_group = school_summary.groupby(["budget_group"])

group_avg_math = school_spending_group["Average Math Score"].mean()
group_avg_read = school_spending_group["Average Reading Score"].mean()
group_avg_math_passing = school_spending_group["% Passing Math"].mean()
group_avg_read_passing = school_spending_group["% Passing Reading"].mean()
group_avg_overall_passing = school_spending_group["Overall Passing Rate"].mean()

school_spending_summary = pd.DataFrame({"Average Math Score":group_avg_math,"Average Reading Score":group_avg_read,
                                       "% Passing Math":group_avg_math_passing,"% Passing Reading":group_avg_read_passing,
                                       "Overall Passing Rate":group_avg_overall_passing})
school_spending_summary = school_spending_summary.style.format("{:.2f}")

school_spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
budget_group,,,,,
<$585,83.46,83.93,93.46,96.61,95.04
$585-615,83.60,83.89,94.23,95.90,95.07
$615-645,79.08,81.89,75.67,86.11,80.89
$645-675,77.00,81.03,66.16,81.13,73.65


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [12]:
school_summary["school size"] = pd.cut(school_summary["Total Students"], size_bins, labels = group_names)

school_size_group = school_summary.groupby(["school size"])

group_avg_math2 = school_size_group["Average Math Score"].mean()
group_avg_read2 = school_size_group["Average Reading Score"].mean()
group_avg_math_passing2 = school_size_group["% Passing Math"].mean()
group_avg_read_passing2 = school_size_group["% Passing Reading"].mean()
group_avg_overall_passing2 = school_size_group["Overall Passing Rate"].mean()

school_size_summary = pd.DataFrame({"Average Math Score":group_avg_math2,"Average Reading Score":group_avg_read2,
                                       "% Passing Math":group_avg_math_passing2,"% Passing Reading":group_avg_read_passing2,
                                       "Overall Passing Rate":group_avg_overall_passing2})
school_size_summary = school_size_summary.style.format("{:.2f}")

school_size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


## Scores by School Type

* Perform the same operations as above, based on school type.

In [13]:
school_type_group = school_summary.groupby(["School Type"])

group_avg_math3 = school_type_group["Average Math Score"].mean()
group_avg_read3 = school_type_group["Average Reading Score"].mean()
group_avg_math_passing3 = school_type_group["% Passing Math"].mean()
group_avg_read_passing3 = school_type_group["% Passing Reading"].mean()
group_avg_overall_passing3 = school_type_group["Overall Passing Rate"].mean()

school_type_summary = pd.DataFrame({"Average Math Score":group_avg_math3,"Average Reading Score":group_avg_read3,
                                       "% Passing Math":group_avg_math_passing3,"% Passing Reading":group_avg_read_passing3,
                                       "Overall Passing Rate":group_avg_overall_passing3})
school_type_summary = school_type_summary.style.format("{:.2f}")

school_type_summary 


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
